In [3]:
import pandas as pd
import statsmodels.api as sm
import scipy.stats as stats

In [14]:
county = 'SB'

In [15]:
df = pd.read_excel('environmental_data.xlsx', sheet_name=county, index_col=0)
df1 = df[['Precipitation', 'Highest Temperature', 'Lowest Temperature', 'Avg Temperature', 'Number of bear encounters']]
df2 = df[['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']]
df.head()

,Precipitation,Highest Temperature,Lowest Temperature,Avg Temperature,Number of bear encounters,anger,disgust,fear,joy,neutral,sadness,surprise
SAN BERNARDINO,,,,,,,,,,,,
Jan,4.26,62,11,36.5,27,6,0,5,5,7,0,4
Feb,3.47,61,10,35.9,17,3,1,2,2,9,0,0
Mar,2.23,66,15,40.2,17,3,2,3,1,8,0,0
Apr,0.83,72,22,46.0,27,2,0,4,7,12,0,2
May,0.50,77,27,50.9,30,2,2,8,3,12,1,2


In [16]:
#Correlation Analysis
df1.corr()

,Precipitation,Highest Temperature,Lowest Temperature,Avg Temperature,Number of bear encounters
Precipitation,1.000000,-0.853657,-0.802227,-0.811166,-0.313074
Highest Temperature,-0.853657,1.000000,0.981878,0.990518,0.637577
Lowest Temperature,-0.802227,0.981878,1.000000,0.995487,0.644899
Avg Temperature,-0.811166,0.990518,0.995487,1.000000,0.636696
Number of bear encounters,-0.313074,0.637577,0.644899,0.636696,1.000000


In [17]:
#Linear Regression
Y = df1['Number of bear encounters']
X = df1['Avg Temperature']
X = sm.add_constant(X)  # Adding a constant (intercept term)

model = sm.OLS(Y, X)
results = model.fit()

print(results.summary())

                                OLS Regression Results                               
Dep. Variable:     Number of bear encounters   R-squared:                       0.405
Model:                                   OLS   Adj. R-squared:                  0.346
Method:                        Least Squares   F-statistic:                     6.818
Date:                       Fri, 23 Jun 2023   Prob (F-statistic):             0.0260
Time:                               03:42:26   Log-Likelihood:                -40.152
No. Observations:                         12   AIC:                             84.30
Df Residuals:                             10   BIC:                             85.27
Df Model:                                  1                                         
Covariance Type:                   nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------

/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [18]:
#Linear Regression
Y = df1['Number of bear encounters']
X = df1['Precipitation']
X = sm.add_constant(X)  # Adding a constant (intercept term)

model = sm.OLS(Y, X)
results = model.fit()

print(results.summary())

                                OLS Regression Results                               
Dep. Variable:     Number of bear encounters   R-squared:                       0.098
Model:                                   OLS   Adj. R-squared:                  0.008
Method:                        Least Squares   F-statistic:                     1.087
Date:                       Fri, 23 Jun 2023   Prob (F-statistic):              0.322
Time:                               03:42:27   Log-Likelihood:                -42.652
No. Observations:                         12   AIC:                             89.30
Df Residuals:                             10   BIC:                             90.27
Df Model:                                  1                                         
Covariance Type:                   nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------

/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [8]:
# df2 = df2.apply(lambda row: row/row.sum(), axis=1)
df2_reset = df2.reset_index()
df2_melted = df2_reset.melt(id_vars=df2_reset.columns[0], var_name='Sentiment', value_name='Count')
df2_melted

,SAN DIEGO,Sentiment,Count
0,Jan,anger,1
1,Feb,anger,2
2,Mar,anger,3
3,Apr,anger,2
4,May,anger,2
...,...,...,...
79,Aug,surprise,1
80,Sep,surprise,1
81,Oct,surprise,3
82,Nov,surprise,3


In [32]:
#One-way ANOVA
groups = df2_melted.groupby('Sentiment')['Count'].apply(list).values.tolist()
f_val, p_val = stats.f_oneway(*groups)

print('Sentiments & Months One-way ANOVA summary')
print('=====================')
print('F value:', f'{f_val:.3f}')
print('p value:', f'{p_val:.3f}')

Sentiments & Months One-way ANOVA summary
F value: 8.112
p value: 0.000


In [33]:
#Correlation Analysis
print(f"Correlation between sentiment (N>30) and Avg Temp:")
sent_1 = []
for sentiment in df2:
    correlation = df2[sentiment].corr(df1['Avg Temperature'])
    if correlation != 1 and sum(df2[sentiment]) > 30:
        sent_1.append((sentiment, correlation))

sent_1 = sorted(sent_1, key=lambda x: -abs(x[1]))
for k, v in sent_1:
    print(f"{k}: {v:.3f}")

Correlation between sentiment (N>30) and Avg Temp:
fear: 0.916
joy: 0.911
neutral: 0.854


In [34]:
#Correlation Analysis
print(f"Correlation between sentiment (N>30) and Precipitation:")
sent_2 = []
for sentiment in df2:
    correlation = df2[sentiment].corr(df1['Precipitation'])
    if correlation != 1 and sum(df2[sentiment]) > 30:
        sent_2.append((sentiment, correlation))

sent_2 = sorted(sent_2, key=lambda x: -abs(x[1]))
for k, v in sent_2:
    print(f"{k}: {v:.3f}")

Correlation between sentiment (N>30) and Precipitation:
fear: -0.799
joy: -0.770
neutral: -0.725
